In [1]:
include("slim_reader.jl")
include("Encoder.jl")
using Knet

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at slim_reader.jl:6
└ @ Core /home/ec2-user/comp492/Text2Image-cemil-slim-dataset-reader/slim_reader.jl:6


In [2]:
BATCH_SIZE = 50
slim_data_dir = "/home/ec2-user/efs/slim-dataset"

"/home/ec2-user/efs/slim-dataset"

In [3]:
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)




Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



2019-11-09 08:48:21.935543: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-09 08:48:21.985206: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300070000 Hz
2019-11-09 08:48:21.985695: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x664bef0 executing computations on platform Host. Devices:
2019-11-09 08:48:21.985719: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-3
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #157: K

Dict{String,Any} with 4 entries:
  "sess"         => PyObject <tensorflow.python.client.session.Session object a…
  "filenames"    => PyObject <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>
  "iterator"     => PyObject <tensorflow.python.data.ops.iterator_ops.Iterator …
  "next_element" => (((PyObject <tf.Tensor 'IteratorGetNext:0' shape=(?, 9, ?, …

In [4]:
# Iterate through the dataset and get the next batch
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);

In [5]:
aimages, acaptions, acameras = getdata(syn_train_batch)

(K32(50,10,128,128,3)[0.6039216⋯], ["There is a magenta cone to the left of a large red capsule.  " "There is a large red capsule behind a magenta cone. The capsule is behind the cone.  " … "There is a magenta cone.  " "There is a magenta cone to the left of a large red capsule.  "; "There is a red ico in front of a violet ico. There is a red ico in front of a large peach box. The ico is to the right of the box. There is a large peach box to the left of a violet ico.  " "There is a red ico in front of a violet ico. There is a red ico to the right of a large peach box. The ico is in front of the box. There is a violet ico to the right of a large peach box.  " … "There is a red ico to the left of a violet ico. There is a red ico to the left of a large peach box. The ico is in front of the box. There is a violet ico in front of a large peach box.  " "There is a red ico in front of a violet ico. There is a large peach box behind a red ico. The box is behind the ico. There is a large peach 

In [6]:
# create a dictionary of known words
vocabid = Dict()
vocabid = Dict("" => 1) # both padding and whitespace KnetArray(zeros(EMBEDDING_SIZE))
vocab = reshape(KnetArray(zeros(EMBEDDING_SIZE)),(EMBEDDING_SIZE,1))

64×1 KnetArray{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
function getembed(vocabid, vocab, w)
    if haskey(vocabid, w)
        return vocab[:,vocabid[w]]
    else
        println("There is no such word in the vocab!!!!")
    end
end

getembed (generic function with 1 method)

In [8]:
function updatevocabs!(vocabid, vocab, w)
    if haskey(vocabid, w)
        println("This word already exists...")
        return vocabid, vocab
    else
        if size(vocab,2) == length(vocabid)
            println("Vocab and vocabid are in order!")
            vocabid[w] = length(vocabid)+1
            vocab = hcat(vocab, KnetArray(Knet.xavier(EMBEDDING_SIZE)))
            return vocabid, vocab
        else
            return nothing
        end
    end    
end

updatevocabs! (generic function with 1 method)

In [9]:
function arrange(batchofcaptions, vocabid, vocab)
    int_batchofcaptions = []
    for i in 1:BATCH_SIZE
        int_captions = []
        captions = batchofcaptions[i,:]
        for c in captions
            int_cap = []
            str = split(c, " ")
            for i in 1:MAX_LENGTH
               if i <= length(str)
                    vocabid, vocab = updatevocabs!(vocabid, vocab, str[i])
                    push!(int_cap, getembed(vocabid, vocab, str[i])) #KnetArray(Knet.xavier(EMBEDDING_SIZE))
                else
                    push!(int_cap, getembed(vocabid, vocab, "")) # padding, same embedding with whitespace
                end
            end
            push!(int_captions, hcat(int_cap...))
        end
        push!(int_batchofcaptions, int_captions)
    end
    return vocabid, vocab, int_batchofcaptions
end

arrange (generic function with 1 method)

In [10]:
vocabid, vocab, caption_embeds = arrange(acaptions, vocabid, vocab)

Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word al

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

...

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

(Dict("pink" => 41,"cone" => 6,"right" => 27,"capsule" => 14,"teal" => 46,"of" => 10,"purple" => 43,"cylinder" => 40,"right." => 45,"to" => 7…), K64(64,48)[0.0⋯], Any[Any[K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯]], Any[K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯]], Any[K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.0716464918374407⋯], K64(64,60)[0.07164649183744

In [12]:
caption_embeds[1]

10-element Array{Any,1}:
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]
 K64(64,60)[0.0716464918374407⋯]

In [13]:
function createconvinput(caption_embeds)
    o = []
    for i in 1:BATCH_SIZE
        push!(o, cat(transpose(caption_embeds[i])...,dims=3))
    end
    return o = cat(o..., dims=3)
end

createconvinput (generic function with 1 method)

In [14]:
input = createconvinput(caption_embeds)

60×64×500 KnetArray{Float64,3}:
[:, :, 1] =
  0.0716465   0.141267    0.13726    …  -0.121747    0.0056529  -0.0241755
  0.161222   -0.0588309  -0.0454987     -0.0598189  -0.129439    0.0211694
  0.114307   -0.0904009  -0.174442       0.111079   -0.147696    0.0117412
  0.0162324   0.057475    0.0336983      0.0813327   0.0545622  -0.0244571
  0.049687   -0.138397    0.0289373     -0.103123   -0.0963927  -0.0840541
 -0.0340697  -0.0817293  -0.0879305  …   0.124844    0.0130495  -0.168097 
  0.0692453  -0.0517244   0.0724719     -0.0575711  -0.146859    0.169462 
  0.143686   -0.0787299   0.0109936      0.129661    0.0800214  -0.0913526
  0.069606   -0.0979674   0.135364       0.129626   -0.0405188   0.0173953
  0.114307   -0.0904009  -0.174442       0.111079   -0.147696    0.0117412
  0.0887982  -0.0671444   0.0677007  …   0.11073    -0.0155936   0.116914 
 -0.156324    0.120884   -0.0882297      0.102648   -0.154235    0.0247394
 -0.0653971  -0.0248078   0.106711       0.170751    0.1

In [223]:
@doc conv4

```
conv4(w, x; kwargs...)
```

Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.

Currently KnetArray{Float32/64,4/5} and Array{Float32/64,4} are supported as `w` and `x`.  If `w` has dimensions `(W1,W2,...,I,O)` and `x` has dimensions `(X1,X2,...,I,N)`, the result `y` will have dimensions `(Y1,Y2,...,O,N)` where

```
Yi=1+floor((Xi+2*padding[i]-Wi)/stride[i])
```

Here `I` is the number of input channels, `O` is the number of output channels, `N` is the number of instances, and `Wi,Xi,Yi` are spatial dimensions.  `padding` and `stride` are keyword arguments that can be specified as a single number (in which case they apply to all dimensions), or an array/tuple with entries for each spatial dimension.

# Keywords

  * `padding=0`: the number of extra zeros implicitly concatenated at the start and at the end of each dimension.
  * `stride=1`: the number of elements to slide to reach the next filtering window.
  * `dilation=1`: dilation factor for each dimension.
  * `mode=0`: 0 for convolution and 1 for cross-correlation.
  * `alpha=1`: can be used to scale the result.
  * `handle`: handle to a previously created cuDNN context. Defaults to a Knet allocated handle.


In [15]:
filter = reshape(KnetArray(Knet.xavier(30*EMBEDDING_SIZE*32)), (1,30,EMBEDDING_SIZE,32)) # output size = embedding/input size
input = reshape(input, (1, MAX_LENGTH, EMBEDDING_SIZE, BATCH_SIZE*NUM_CAPTIONS_PER_SCENE))

1×60×64×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 0.0716465  0.161222  0.114307  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.141267  -0.0588309  -0.0904009  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.13726  -0.0454987  -0.174442  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 1] =
 -0.121747  -0.0598189  0.111079  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 1] =
 0.0056529  -0.129439  -0.147696  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 64, 1] =
 -0.0241755  0.0211694  0.0117412  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.0716465  0.161222  0.114307  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.141267  -0.0588309  -0.0904009  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 2] =
 0.13726  -0.0454987  -0.174442  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 2] =
 -0.121747  -0.0598189  0.111079  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 2] =
 0.0056529  -0.129439  -0.147696  …  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [16]:
@show y = conv4(filter, input, dilation=2)

y = conv4(filter, input, dilation=2) = K64(1,2,32,500)[-0.01859228451980576⋯]


1×2×32×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.0185923  0.000637945

[:, :, 2, 1] =
 -0.0114183  -0.0103292

[:, :, 3, 1] =
 -0.000219923  0.000316161

...

[:, :, 30, 1] =
 0.00735944  0.0038089

[:, :, 31, 1] =
 -0.0159452  0.0060926

[:, :, 32, 1] =
 -0.00167574  0.0179139

[:, :, 1, 2] =
 -0.002521  -0.0108783

[:, :, 2, 2] =
 -0.00643082  0.000426505

[:, :, 3, 2] =
 0.0003341  -0.00893763

...

[:, :, 30, 2] =
 -0.00456749  -0.00302805

[:, :, 31, 2] =
 -0.0142547  0.00915151

[:, :, 32, 2] =
 0.0052396  0.016439

[:, :, 1, 3] =
 -0.0117302  -0.00494047

[:, :, 2, 3] =
 -0.0131273  -0.00684678

[:, :, 3, 3] =
 -0.000890971  0.000181952

...

[:, :, 30, 3] =
 0.00817311  -0.00110635

[:, :, 31, 3] =
 -0.00423575  0.00108984

[:, :, 32, 3] =
 -0.00322948  0.00860288

...

[:, :, 1, 498] =
 -0.0120483  -0.00386516

[:, :, 2, 498] =
 -0.0195812  -0.0111137

[:, :, 3, 498] =
 0.0154741  -0.0034898

...

[:, :, 30, 498] =
 0.0142276  -0.0118502

[:, :, 31, 498] =
 -0.0225821  -0.0

In [17]:
function build_angles(tuplebatch)
    output = []
    for i in 1:BATCH_SIZE
        for j in 1:NUM_CAPTIONS_PER_SCENE
            sin_value = tuplebatch[i,j,3]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            cos_value = tuplebatch[i,j,1]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            push!(output, KnetArray([sin_value, cos_value])) 
        end
    end
    
    return cat(output..., dims=2)
end

build_angles (generic function with 1 method)

In [18]:
tuples = build_angles(acameras)

2×500 KnetArray{Float32,2}:
 0.591685   0.907955   0.46885    0.719238  …   0.370463  0.900371  0.304245
 0.806169  -0.419067  -0.883278  -0.694764     -0.928847  0.435124  0.952594

In [19]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

In [20]:
struct Layer0; w; b; end
Layer0(ir::Int, ic::Int, o::Int) = Layer0(param(o,ir),param0(o, ic))
(l::Layer0)(x) = (l.w * x .+ l.b)

In [21]:
mlp1=Chain(Layer0(2, 500, 32)) # MLP1 dimensionality 32

Chain((Layer0(P(KnetArray{Float32,2}(32,2)), P(KnetArray{Float32,2}(32,500))),))

In [37]:
mlp2=Chain(Layer0(96, 500, 128), Layer0(128, 500, 196), Layer0(196, 500, 256)) # MLP2 dimensionality 256

Chain((Layer0(P(KnetArray{Float32,2}(128,96)), P(KnetArray{Float32,2}(128,500))), Layer0(P(KnetArray{Float32,2}(196,128)), P(KnetArray{Float32,2}(196,500))), Layer0(P(KnetArray{Float32,2}(256,196)), P(KnetArray{Float32,2}(256,500)))))

In [34]:
cameras_hat = mlp1(tuples)

32×500 KnetArray{Float32,2}:
  0.0745257    0.19426      0.132485   …   0.151597      0.0123633 
 -0.124225     0.20713      0.267121       0.00109102   -0.193102  
 -0.0094089    0.299023     0.28063        0.135514     -0.106677  
  0.0348064   -0.074654    -0.0901176     -0.00824912    0.0595041 
 -0.0510609   -0.129174    -0.087166      -0.101991     -0.00974511
 -0.0215605    0.227074     0.222015   …   0.0915455    -0.0956084 
 -0.271668    -0.0734919    0.100322      -0.249261     -0.251254  
  0.105464    -0.0502575   -0.111356       0.0591056     0.123136  
 -0.00634492  -0.246198    -0.222348      -0.122682      0.0735604 
  0.100611     0.00156661  -0.0607278      0.0800516     0.101384  
  0.124212    -0.133165    -0.199136   …   0.0342539     0.169059  
  0.0462653   -0.113046    -0.132482      -0.017568      0.0835819 
 -0.17329      0.103733     0.202412      -0.0870062    -0.2092    
  ⋮                                    ⋱                           
  0.0630812    0.18

In [35]:
captions_hat = convert(KnetArray{Float32}, reshape(reshape(y, (2, 32, 500)), (64, 500)))
#cameras_hat = reshape(cameras_hat, (1, 32, 500))

64×500 KnetArray{Float32,2}:
 -0.0185923    -0.002521     -0.0117302    …  -0.00878594   -0.0122542  
  0.000637945  -0.0108783    -0.00494047       0.000544482   0.00100992 
 -0.0114183    -0.00643082   -0.0131273       -0.00145262   -0.00331303 
 -0.0103292     0.000426505  -0.00684678       0.00923914   -0.00606124 
 -0.000219923   0.0003341    -0.000890971     -0.00222346   -0.00462954 
  0.000316161  -0.00893763    0.000181952  …  -0.000111342  -0.00902352 
  0.00117551    0.0139623     0.00571471      -0.00314144    0.00845862 
  0.00664318   -0.00806741    0.00260579      -0.00441931   -0.00424698 
  0.00800745   -0.00203521    0.00280901      -0.00466727   -0.00066008 
 -0.00658675   -0.0110613    -0.0072011       -0.0101732    -0.0185337  
  0.00984879   -0.00976052   -0.00546583   …  -0.0126326    -0.00803725 
 -0.00206585    0.011752     -0.00120287       0.000379708  -0.00157411 
  0.00567781   -0.0064058     0.00249578       0.000174517   0.014699   
  ⋮                   

In [39]:
# dimension düzenleeee 3x32x500 girsin 256 çıksın magicc
h = mlp2(cat(captions_hat, cameras_hat, dims=1))

256×500 KnetArray{Float32,2}:
 -0.00255739    0.0256916     0.0244907    …   0.011673     -0.0091049  
  0.00447942    0.0142129     0.00966975       0.0114714     0.00137569 
  0.00472452   -0.00732957   -0.00938582       0.000607332   0.00772291 
 -0.00868309    0.0015345     0.00722403      -0.00580782   -0.00925642 
 -0.0149848    -0.0112605    -0.0018835       -0.0168299    -0.0112473  
  0.000234584   0.000135116  -0.000410786  …   0.000710357  -0.000389661
  0.00572256    0.00244949   -0.00282629       0.00552148    0.00548284 
 -0.00347227    0.0045721     0.00656647      -0.000458508  -0.00400226 
  0.0180878    -0.000461502  -0.0122899        0.013992      0.017103   
 -0.0192294    -0.00628284    0.00648228      -0.0198352    -0.0174253  
  0.0041211     0.0146247     0.0119404    …   0.0112687     0.000245617
 -0.00317035   -0.0183934    -0.014274        -0.0113522     0.00147021 
 -0.0188335     0.0140987     0.0253936       -0.00790689   -0.0253626  
  ⋮                  

In [60]:
function aggregate(h) # batch of h s
    r = []
    for i in 1:BATCH_SIZE
        push!(r, sum(h[:,10*(i-1)+1:10*i-1], dims=2)./(NUM_CAPTIONS_PER_SCENE-1)) # 1...9, 11...19, ...
    end
    return hcat(r...)
end

aggregate (generic function with 1 method)

In [61]:
aggregate(h)

256×50 KnetArray{Float32,2}:
  0.0173372     0.0188766     0.00409929   …   0.00509701    0.00346261 
  0.00932002    0.0092528     0.00472913       0.00571502    0.00521986 
 -0.0049831    -0.00654657   -0.000274951      0.00090942    0.000913373
  0.00133586    0.00261282   -0.00256112      -0.00389133   -0.00362298 
 -0.00744695   -0.00557629   -0.00687771      -0.00984633   -0.00939304 
  0.000172507  -0.000120807   0.000665638  …   0.00101492    0.000887494
  0.00107955   -0.000608547   0.00219268       0.00311126    0.00309379 
  0.00328585    0.00349771    0.000244146     -0.000120456  -0.000313044
 -0.00107827   -0.00556524    0.00489438       0.008805      0.00804703 
 -0.00374312   -0.000647311  -0.00686463      -0.0119583    -0.0107329  
  0.0101313     0.0103755     0.00446604   …   0.00619833    0.00596174 
 -0.012203     -0.0126629    -0.00463436      -0.00604339   -0.00499485 
  0.0103789     0.0154273    -0.00297272      -0.00609701   -0.00599714 
  ⋮                   

In [59]:
sum(dene[:,1:9], dims=2)./(NUM_CAPTIONS_PER_SCENE-1)

256×1 KnetArray{Float32,2}:
  0.017337192  
  0.009320018  
 -0.0049831034 
  0.0013358599 
 -0.0074469466 
  0.00017250732
  0.0010795472 
  0.003285852  
 -0.0010782653 
 -0.003743117  
  0.010131344  
 -0.012202985  
  0.0103788795 
  ⋮            
  0.0018463805 
  0.003984873  
 -0.007729738  
  0.007007519  
  0.013330829  
  0.010734635  
 -0.006642709  
  0.011024526  
  0.000637059  
 -0.012073035  
  0.012258187  
  0.0061029573 